# Universidad del Valle de Guatemala
## Security Data Science - 10
* Juan Manuel Marroquin Alfaro - 19845
* Carlos Alberto Raxtum Ramos - 19721
* Jose Abraham Gutierrez Corado - 19111
* Walter Danilo Saldaña Salguero - 19897
* Javier Alejandro Cotto Argueta - 19324

## Proyecto Fase 2: Selección de caracteristicas y limpieza de datos

In [10]:
from PIL import Image
import numpy as np
import sys
import os
import csv# default format can be changed as needed
def createFileList(myDir, format='.pgm'):
    fileList = []
    print(myDir)
    labels = []
    names = []
    keywords = {"K" : "1",} # keys and values to be changed as needed
    for root, dirs, files in os.walk(myDir, topdown=True):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
            for keyword in keywords:
                if keyword in name:
                    labels.append(keywords[keyword])
                else:
                    continue
            names.append(name)
    return fileList, labels, names

myFileList, labels, names  = createFileList('./BOSSbase/BOSSbase_1.01')
print(labels)
i = 0
for file in myFileList:
    print(file)
    img_file = Image.open(file)
    # img_file.show()# get original image parameters...
    width, height = img_file.size
    format = img_file.format
    mode = img_file.mode# Make image Greyscale
    img_grey = img_file.convert('L')
    #img_grey.save('result.png')
    #img_grey.show()# Save Greyscale values
    value = np.asarray(img_grey.getdata(), dtype=int).reshape((width, height))
    value = value.flatten()
    
    value = np.append(value,labels[i])
    i +=1
    
    print(value)
    with open("images-db.csv", 'a') as f:
        writer = csv.writer(f)
        writer.writerow(value)

./BOSSbase/BOSSbase_1.01
[]
./BOSSbase/BOSSbase_1.01\1.pgm


IndexError: list index out of range